In [2]:
import yaml
import pandas as pd
import numpy as np
import os
import scipy.stats
import collections

pd.set_option("display.min_rows", 50)

### Create cross validation dataframe

In [3]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec19-21")
header = ["preprocess", "bert_lr", "model_lr", "warmup", "movie", "dev_lea", "train_lea", "best_epoch", "dev_scores",
          "train_scores"]
rows = []

for dir_ in os.listdir(results_dir):
    result_file1 = os.path.join(results_dir, dir_, "result.yaml")
    result_file2 = os.path.join(results_dir, dir_, "result2.yaml")
    if os.path.exists(result_file1):
        with open(result_file1) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    elif os.path.exists(result_file2):
        with open(result_file2) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    else:
        continue
    preprocess = result["preprocess"]
    bert_lr = result["bert_lr"]
    model_lr = result["coref_lr"]
    warmup = result["warmup_steps"]
    movie = result["test_movie"]
    dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
    epoch = result["best_epoch"]
    dev_scores = result["dev_scores"]
    train_scores = result["train_scores"]
    train_lea = train_scores[epoch - 1]
    rows.append([preprocess, bert_lr, model_lr, warmup, movie, dev_lea, train_lea, epoch, dev_scores, train_scores])

df = pd.DataFrame(rows, columns=header)
print(df.shape)
print()

for col, dtype in zip(df.columns, df.dtypes):
    print(f"{col} ({dtype})")

(648, 10)

preprocess (object)
bert_lr (float64)
model_lr (float64)
warmup (float64)
movie (object)
dev_lea (float64)
train_lea (float64)
best_epoch (int64)
dev_scores (object)
train_scores (object)


### Find the macro scores (averaged across movies) for cross validation

In [ ]:
print("macro average:")
macro_df = (df.groupby(["preprocess", "bert_lr", "model_lr", "warmup"])
            .agg({"dev_lea": "mean", "train_lea": "mean", "best_epoch": "mean"})
            .sort_values(by="dev_lea", ascending=False))
display(macro_df)

macro average:


dev_lea  train_lea  best_epoch
preprocess   bert_lr model_lr warmup                                  
regular      0.00002 0.0002    50.0   61.600000  89.835683    6.833333
addsays      0.00002 0.0001   -1.0    61.592167  89.766367    6.333333
             0.00001 0.0002   -1.0    61.572500  90.609750    8.166667
regular      0.00002 0.0002    100.0  61.405833  88.221367    8.166667
addsays      0.00002 0.0002    50.0   61.316833  86.292750    5.666667
                     0.0001    0.0    61.287667  88.543567    7.166667
regular      0.00005 0.0002    50.0   61.252167  89.076450    7.666667
addsays      0.00002 0.0005   -1.0    61.202667  90.920317    6.833333
             0.00005 0.0001    50.0   60.993833  93.758217   11.500000
regular      0.00002 0.0005    0.0    60.984667  90.512150    7.000000
addsays      0.00002 0.0005    0.0    60.979667  90.086967    5.833333
                     0.0002    0.0    60.945833  89.713150    6.666667
regular      0.00005 0.0002    100.0  60.909167  88.606300    6.833333
addsays      0.00002 0.0001    100.0  60.897000  88.548567    6.833333
                     0.0005    100.0  60.868333  90.499900    7.666667
regular      0.00001 0.0005    100.0  60.863167  89.608933    9.500000
addsays      0.00005 0.0002    50.0   60.837833  94.402283   11.166667
             0.00001 0.0001   -1.0    60.783333  85.620067    5.333333
             0.00005 0.0005    0.0    60.764167  92.057350    8.666667
             0.00002 0.0005    50.0   60.707167  87.971267    4.666667
regular      0.00002 0.0005    50.0   60.688333  92.091100    9.333333
addsays      0.00001 0.0005    0.0    60.687833  89.505417    7.000000
regular      0.00001 0.0002    100.0  60.685667  90.352500   11.833333
                              -1.0    60.642000  89.331383    7.666667
addsays      0.00005 0.0001   -1.0    60.625000  92.397300    9.833333
...                                         ...        ...         ...
nocharacters 0.00001 0.0002    50.0   52.090167  78.526783    8.333333
             0.00002 0.0005    100.0  51.958500  83.161617    9.500000
                               0.0    51.925167  81.388083    6.666667
                               50.0   51.916667  82.716600    8.833333
                     0.0002   -1.0    51.915333  82.109617    7.166667
             0.00005 0.0002    50.0   51.853500  80.220650    8.000000
             0.00001 0.0002   -1.0    51.705500  81.751600    8.500000
             0.00005 0.0005    50.0   51.649167  83.518117    8.333333
             0.00001 0.0005   -1.0    51.616667  82.128800    8.333333
                               50.0   51.537667  76.588583    8.166667
                     0.0002    0.0    51.502500  81.723817   10.333333
                     0.0005    100.0  51.440000  76.558117    8.333333
             0.00002 0.0001    0.0    51.398167  78.699733    5.666667
                               50.0   51.376833  74.958800    5.333333
             0.00005 0.0001    100.0  51.328667  73.792083    4.666667
                              -1.0    51.236500  80.234217    8.000000
             0.00001 0.0002    100.0  51.056833  73.031850    6.500000
                     0.0001    50.0   51.035833  72.434417    5.000000
                     0.0005    0.0    50.959167  81.468817    8.166667
             0.00005 0.0002    0.0    50.773000  77.587600    9.166667
             0.00001 0.0001    100.0  50.719000  71.431000    7.666667
                               0.0    50.707000  80.217683    8.833333
             0.00005 0.0005    0.0    50.126167  80.046167    7.666667
                     0.0002   -1.0    49.846167  82.848767    8.833333
                     0.0001    0.0    48.236667  73.919417    5.833333

[108 rows x 3 columns]

### Save the cross validation dataframe

In [9]:
df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/cross_val_1.csv"),
          index=False)

In [6]:
# mean per movie
df.groupby("movie").agg({"dev_lea": "mean", "train_lea": "mean", "best_epoch": "mean"})

,dev_lea,train_lea,best_epoch
movie,,,
avengers_endgame,43.708306,88.734183,8.472222
dead_poets_society,55.264519,86.943589,8.212963
john_wick,67.745519,87.336787,9.342593
prestige,55.560056,83.633989,6.314815
quiet_place,60.651444,84.949249,7.870370
zootopia,61.659074,86.566273,8.444444


### Read baseline dataframe for full-length scripts

In [7]:
baseline_df = pd.read_csv(os.path.join(os.getenv("DATA_DIR"),
                                       "mica_text_coref/movie_coref/results/coreference/full_length.baseline.tsv"),
                          sep="\t", index_col=None)
print(baseline_df.shape)
print()

for col, dtype in zip(baseline_df.columns, baseline_df.dtypes):
    print(f"{col} ({dtype})")

(254016, 14)

preprocess (object)
genre (object)
split_len (int64)
overlap_len (int64)
merge_strategy (object)
merge_speakers (bool)
entity (object)
remove_gold_singletons (bool)
provide_gold_mentions (bool)
movie (object)
metric (object)
P (float64)
R (float64)
F (float64)


In [8]:
# summary of columns
print(baseline_df["preprocess"].unique())
print(baseline_df["genre"].unique())
print(baseline_df["entity"].unique())
print(baseline_df["split_len"].unique())
print(baseline_df["overlap_len"].unique())
print(baseline_df["merge_strategy"].unique())

['nocharacters' 'addsays' 'none']
['mz' 'tc' 'wb' 'bc' 'pt' 'nw' 'bn']
['all' 'speaker' 'person']
[3072 2048 5120 4096]
[256 128 512]
['pre' 'post' 'avg' 'max' 'min' 'none']


### Find the macro baseline scores (averaged across movies)

In [9]:
macro_baseline_df = (baseline_df[(baseline_df["movie"] != "all")
                                 & ~baseline_df["remove_gold_singletons"]
                                 & ~baseline_df["provide_gold_mentions"]
                                 & (baseline_df["metric"] == "lea")]
                     .groupby(["preprocess", "entity", "genre", "split_len", "overlap_len", "merge_strategy",
                               "merge_speakers"])
                     .agg({"F": "mean"})
                     .sort_values(by="F", ascending=False))
display(macro_baseline_df)

F
preprocess   entity  genre split_len overlap_len merge_strategy merge_speakers           
addsays      speaker wb    3072      512         avg            True            59.432041
                                                 min            True            59.413063
                                                 post           True            59.381315
                           5120      128         none           True            59.241314
                     tc    3072      512         avg            True            59.165600
                     mz    5120      128         none           True            59.056600
                     tc    3072      512         post           True            59.033369
                     wb    5120      128         max            True            58.976455
                           3072      512         max            True            58.968410
                           5120      128         avg            True            58.966080
                                                 post           True            58.962806
                                                 pre            True            58.956274
                                                 min            True            58.948239
                                     512         post           True            58.854751
                                                 max            True            58.840830
                     mz    5120      128         max            True            58.787151
                                                 avg            True            58.778891
                                                 post           True            58.775623
                                                 pre            True            58.766965
                                                 min            True            58.761043
                     wb    5120      512         pre            True            58.635215
                                                 none           True            58.627075
                     tc    3072      512         max            True            58.581506
                     mz    5120      512         post           True            58.489019
                     tc    3072      512         min            True            58.421999
...                                                                                   ...
nocharacters speaker bn    2048      256         post           False            2.352219
                     pt    2048      256         post           False            2.352148
                     nw    2048      256         post           False            2.350901
                     tc    2048      256         max            False            2.336115
                     pt    2048      256         max            False            2.327803
                     bn    2048      256         max            False            2.325054
                     nw    2048      256         max            False            2.324603
                     mz    2048      256         max            False            2.322912
                     wb    2048      256         post           False            2.318539
                                                 max            False            2.313053
                     bn    2048      256         avg            False            2.288220
                     pt    2048      256         avg            False            2.281867
                     tc    2048      256         none           True             2.239559
                     bn    2048      256         none           True             2.235082
                     nw    2048      256         none           True             2.234590
                     pt    2048      256         none           True             2.234558
                     mz    2048      256         none           True             2.226877
                     wb    2048      2

In [10]:
hyperparam_to_movie_scores = {}
for (preprocess, bert_lr, model_lr, warmup), gdf in df.groupby(["preprocess", "bert_lr", "model_lr", "warmup"]):
    gdf.sort_values(by="movie", inplace=True)
    hyperparam_to_movie_scores[(preprocess, bert_lr, model_lr, warmup)] = gdf["dev_lea"].tolist()

In [12]:
hyperparams = sorted(hyperparam_to_movie_scores.keys(), key=lambda hp: np.mean(hyperparam_to_movie_scores[hp]),
                     reverse=True)
print(hyperparams[:10])

[('regular', 2e-05, 0.0002, 50.0), ('addsays', 2e-05, 0.0001, -1.0), ('addsays', 1e-05, 0.0002, -1.0), ('regular', 2e-05, 0.0002, 100.0), ('addsays', 2e-05, 0.0002, 50.0), ('addsays', 2e-05, 0.0001, 0.0), ('regular', 5e-05, 0.0002, 50.0), ('addsays', 2e-05, 0.0005, -1.0), ('addsays', 5e-05, 0.0001, 50.0), ('regular', 2e-05, 0.0005, 0.0)]


In [18]:
rows = [list(hyperparams[0]) + [np.mean(hyperparam_to_movie_scores[hyperparams[0]]), 0, 0]]
rank = 0
for i in range(len(hyperparams) - 1):
    arr1 = hyperparam_to_movie_scores[hyperparams[0]]
    arr2 = hyperparam_to_movie_scores[hyperparams[i + 1]]
    ttest_result = scipy.stats.ttest_rel(arr1, arr2, alternative="greater")
    if ttest_result.pvalue < 0.05:
        rank += 1
    rows.append(list(hyperparams[i + 1]) + [np.mean(arr2), ttest_result.pvalue, rank])
rank_df = pd.DataFrame(rows, columns=["preprocess", "bert_lr", "model_lr", "warmup", "macro_score", "pvalue", "rank"])
rank_df

,preprocess,bert_lr,model_lr,warmup,macro_score,pvalue,rank
0,regular,0.00002,0.0002,50.0,61.600000,0.000000,0
1,addsays,0.00002,0.0001,-1.0,61.592167,0.499146,0
2,addsays,0.00001,0.0002,-1.0,61.572500,0.496678,0
3,regular,0.00002,0.0002,100.0,61.405833,0.409886,0
4,addsays,0.00002,0.0002,50.0,61.316833,0.463974,0
5,addsays,0.00002,0.0001,0.0,61.287667,0.462817,0
6,regular,0.00005,0.0002,50.0,61.252167,0.416443,0
7,addsays,0.00002,0.0005,-1.0,61.202667,0.452464,0
8,addsays,0.00005,0.0001,50.0,60.993833,0.430795,0
9,regular,0.00002,0.0005,0.0,60.984667,0.287324,0


In [19]:
rank_df.to_csv(os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/temp/rank.csv"), index=False)

In [26]:
collections.Counter(rank_df.loc[rank_df["rank"] == 0, "bert_lr"])
collections.Counter(rank_df.loc[rank_df["rank"] == 0, "model_lr"])

Counter({0.0002: 12, 0.0001: 9, 0.0005: 15})

In [4]:
results_dir = os.path.join(os.getenv("DATA_DIR"),
                           "mica_text_coref/movie_coref/results/coreference/cross_val_excerpts_Dec25-26")
header = ["weight_decay", "dropout", "movie", "dev_lea", "best_epoch"]
rows = []

for dir_ in os.listdir(results_dir):
    result_file = os.path.join(results_dir, dir_, "result.yaml")
    if os.path.exists(result_file):
        with open(result_file) as f:
            result = yaml.load(f, Loader=yaml.FullLoader)
    else:
        continue
    weight_decay = result["weight_decay"]
    dropout = result["dropout"]
    movie = result["test_movie"]
    dev_lea = result["dev_metric"]["span"]["lea"]["f1"]
    epoch = result["best_epoch"]
    rows.append([weight_decay, dropout, movie, dev_lea, epoch])

df2 = pd.DataFrame(rows, columns=header)
print(df2.shape)
print()

for col, dtype in zip(df2.columns, df2.dtypes):
    print(f"{col} ({dtype})")

(194, 5)

weight_decay (float64)
dropout (float64)
movie (object)
dev_lea (float64)
best_epoch (int64)


In [5]:
macro_df2 = df2.groupby(["weight_decay", "dropout"]).agg({"movie": "count", "dev_lea": "mean"})
macro_df2 = macro_df2[macro_df2["movie"] == 6].sort_values(by="dev_lea", ascending=False)
macro_df2

movie    dev_lea
weight_decay dropout                  
0.0010       0.0          6  61.600000
0.0100       0.0          6  61.085833
1.0000       0.0          6  61.080167
0.1000       0.0          6  60.119500
0.0000       0.0          6  59.836667
0.0001       0.0          6  59.836667
10.0000      0.0          6  59.632000
             0.2          6  58.728167
0.1000       0.2          6  57.957000
1.0000       0.2          6  57.753333
0.0010       0.2          6  57.636167
0.0100       0.2          6  57.298167
0.1000       0.6          6  55.452000
10.0000      0.4          6  55.423333
1.0000       0.4          6  55.268000
0.0010       0.6          6  55.010667
             0.4          6  54.758833
1.0000       0.6          6  54.736333
0.1000       0.4          6  54.708500
10.0000      0.6          6  54.535333
0.0001       0.4          6  54.476167
0.0000       0.4          6  54.476167
0.0100       0.4          6  54.220000
             0.6          6  54.097167
0.0001       0.6          6  54.019667
0.0000       0.6          6  54.019667
100.0000     0.6          6  48.038000
             0.8          6   0.000000